<a href="https://colab.research.google.com/github/parangatm/IoT-Security/blob/main/CA4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EC209AS - Computer Assignment 4

In [ ]:
## Importing the necessary libraries
import hashlib
import random
import time

In [ ]:
## Reading the necessary files
rn_16 = open("random_numbers_16.txt", "r")
at_8 = open("attack_8.txt", "r")
zr_8 = open("zeros_8.txt", "r")

mem_contents = rn_16.read().split('\n')
attack_contents = at_8.read().split('\n')
zero_contents = zr_8.read().split('\n')

### Design

1. **Verifier**: Write a verifier function that creates a nonce, sends the nonce to the prover function, and invokes it (details later). Once the response is received, it locally computes the hash itself and compares it with the response. It passes the test if the two hashes are equal.

2. **Prover**: It takes the nonce, reads the memory line-by-line, and computes the hash. It returns the final hash value.

In [ ]:
## verifier function
def verifier(mem_contents, prover):
    nonce = random.randint(0, 256)
    start_time = time.time()
    response_hash = prover(nonce, mem_contents)
    end_time = time.time()
    time_elapsed = end_time - start_time
    g = hashlib.sha256()
    g.update(str(nonce).encode())
    for line in mem_contents:
        g.update(line.encode())
    golden_hash = g.hexdigest()
    verified = (golden_hash == response_hash)
    return verified, time_elapsed

In [ ]:
## prover function
def prover(nonce, mem_contents):
    m = hashlib.sha256()
    m.update(str(nonce).encode())
    for line in mem_contents:
        m.update(line.encode())
    computed_hash = m.hexdigest()
    return computed_hash

### Step 1
Measure the time it takes to complete an attestation. Use a timer and start it after creating the nonce and stop it once the prover function returns its final hash (i.e., measure how long it takes to compute the hash in prover). To find the average time, repeat this process 10 times and report the average.

In [ ]:
## Averaging over 10 iterations
times = []
for _ in range(10):
    authenticated, time_taken = verifier(mem_contents, prover)
    times.append(time_taken)

avg_time = sum(times)/len(times)
print(avg_time)

0.0003537416458129883


### Step 2
Create an attack as follows:
- Create a pre-compute function that does the following: it combines the “_zeros_8_” and “_attack_8_” files by copying the non-zero parts of the former into the zero parts of the latter.
- Use the new file, write a malicous_prover function that takes the nonce and computes the correct hash for the zeros_8 file but only using the attack_8 file (note that you can assume that the second half of the file should have been zero, and use that to forward the correct content and compute the hash).

In [ ]:
## Pre-compute function
def pre_compute(attack_contents, zero_contents):
    new_contents = []
    for attack_line, zero_line in zip(attack_contents, zero_contents):
        if attack_line == "0":
            new_contents.append(zero_line)
        else:
            new_contents.append(attack_line)
    with open("out.txt", "w") as f:
        f.writelines(new_contents)
    return new_contents

new_contents = pre_compute(attack_contents, zero_contents)

In [ ]:
## Malicious Prover
def malicious_prover(nonce,actual_contents,attack_contents=attack_contents):
    new_contents = []
    m = hashlib.sha256()
    m.update(str(nonce).encode())
    for actual_line, attack_line in zip(actual_contents, attack_contents):
        m.update(actual_line.encode())
        if actual_line == "0":
            new_contents.append(attack_line)
        else:
            new_contents.append(actual_line)
    computed_hash = m.hexdigest()
    return computed_hash

In [ ]:
authenticated, time_taken = verifier(zero_contents, malicious_prover)
print(authenticated)

True


### Step 3
Once you confirm that the attack passes the test successfully, measure the time for the malicous_prover function. Report the average (10 runs).

In [ ]:
true_times = []
for _ in range(10):
    authenticated, time_taken = verifier(zero_contents, prover)
    true_times.append(time_taken)

malicious_times = []
for _ in range(10):
    authenticated, time_taken = verifier(zero_contents, malicious_prover)
    malicious_times.append(time_taken)

avg_true_time = sum(true_times)/len(true_times)
print("Average true time: ", avg_true_time)

avg_malicious_time = sum(malicious_times)/len(malicious_times)
print("Average malicious time: ", avg_malicious_time)

Average true time:  0.0001874208450317383
Average malicious time:  0.000377964973449707


In [ ]:
print([i*100000 for i in true_times])

[23.2696533203125, 16.379356384277344, 16.379356384277344, 16.21246337890625, 24.509429931640625, 22.530555725097656, 16.069412231445312, 20.742416381835938, 15.664100646972656, 15.664100646972656]


In [ ]:
print([i*100000 for i in malicious_times])

[30.58910369873047, 28.848648071289062, 28.443336486816406, 28.53870391845703, 28.276443481445312, 31.35204315185547, 52.21366882324219, 52.97660827636719, 46.944618225097656, 49.78179931640625]


### Step 4
Assuming that the times are slightly different, modify the verifier function to check the response time and only accept the response only if the hash is correct and the response time is less than a threshold. Explain how you find that threshold.

In [ ]:
## Verifier with a threshold
def verifier_threshold(mem_contents, prover):
    nonce = random.randint(0, 256)
    start_time = time.time()
    response_hash = prover(nonce, mem_contents)
    end_time = time.time()
    time_elapsed = end_time - start_time
    g = hashlib.sha256()
    g.update(str(nonce).encode())
    for line in mem_contents:
        g.update(line.encode())
    golden_hash = g.hexdigest()
    if golden_hash == response_hash:
        if (time_elapsed*100000) < 28.0:
            return True, time_elapsed
    return False, time_elapsed

**Choice for Threshold** : Based on observing the true and malicious prover times for a few iterations, 0.00028 is a valid threshold with **most** malicious prover functions not being authenticated.

In [ ]:
auth, time_true = verifier_threshold(zero_contents, prover)
print(auth, time_true)

m_auth, time_m = verifier_threshold(zero_contents, malicious_prover)
print(m_auth, time_m)

True 0.00020241737365722656
False 0.0006394386291503906


### Step 5
Repeat this process 10 times, 5 with the correct verifier and 5 with the malicious, and report the true and false positive rates.

In [ ]:
true_times = []
true_responses = []
for _ in range(10):
    authenticated, time_taken = verifier_threshold(zero_contents, prover)
    true_times.append(time_taken)
    true_responses.append(authenticated)

malicious_times = []
malicious_responses = []
for _ in range(10):
    authenticated, time_taken = verifier_threshold(zero_contents, malicious_prover)
    malicious_times.append(time_taken)
    malicious_responses.append(authenticated)

avg_true_time = sum(true_times)/len(true_times)
true_positive = true_responses.count(True)/len(true_responses)
avg_malicious_time = sum(malicious_times)/len(malicious_times)
false_positive = malicious_responses.count(True)/len(malicious_responses)

print("Average true time: ", avg_true_time*100000)
print("Average malicious time: ", avg_malicious_time*100000)

print("True positive: ", true_positive)
print("False positive: ", false_positive)

Average true time:  19.55270767211914
Average malicious time:  41.544437408447266
True positive:  0.8
False positive:  0.3


### Step 6
Add a random noise to the timer (to mimic the communication and computation randomness). Report the amount of noise required to reduce the true positive rate to 50%.

In [ ]:
## Verifier with a threshold
def verifier_threshold_noise(mem_contents, prover, noise):
    nonce = random.randint(0, 256)
    start_time = time.time()
    response_hash = prover(nonce, mem_contents)
    end_time = time.time()
    random_noise = noise
    time_elapsed = end_time - start_time + random_noise
    g = hashlib.sha256()
    g.update(str(nonce).encode())
    for line in mem_contents:
        g.update(line.encode())
    golden_hash = g.hexdigest()
    if golden_hash == response_hash:
        if (time_elapsed*100000) < 28.0:
            return True, time_elapsed
    return False, time_elapsed

In [ ]:
noise = 0.00002

true_responses = []
for _ in range(10):
    authenticated, time_taken = verifier_threshold_noise(zero_contents, prover, noise)
    true_responses.append(authenticated)

malicious_responses = []
for _ in range(10):
    authenticated, time_taken = verifier_threshold_noise(zero_contents, malicious_prover, noise)
    malicious_responses.append(authenticated)

true_positive = true_responses.count(True)/(len(true_responses)+len(malicious_responses))
print("True positive: ", true_positive)

True positive:  0.35


Finding the **optimum noise** - iterating through a range of values multiple times

In [ ]:
for _ in range(5):
    for n in range(9):
        noise = n / 100000

        true_responses = []
        for _ in range(10):
            authenticated, time_taken = verifier_threshold_noise(zero_contents, prover, noise)
            true_responses.append(authenticated)

        malicious_responses = []
        for _ in range(10):
            authenticated, time_taken = verifier_threshold_noise(zero_contents, malicious_prover, noise)
            malicious_responses.append(authenticated)

        true_positive = true_responses.count(True)/(len(true_responses)+len(malicious_responses))
        print("Noise: ", noise, " True positive: ", true_positive)
    print()

Noise:  0.0  True positive:  0.4
Noise:  1e-05  True positive:  0.15
Noise:  2e-05  True positive:  0.5
Noise:  3e-05  True positive:  0.25
Noise:  4e-05  True positive:  0.2
Noise:  5e-05  True positive:  0.5
Noise:  6e-05  True positive:  0.5
Noise:  7e-05  True positive:  0.5
Noise:  8e-05  True positive:  0.35

Noise:  0.0  True positive:  0.5
Noise:  1e-05  True positive:  0.4
Noise:  2e-05  True positive:  0.5
Noise:  3e-05  True positive:  0.3
Noise:  4e-05  True positive:  0.5
Noise:  5e-05  True positive:  0.5
Noise:  6e-05  True positive:  0.4
Noise:  7e-05  True positive:  0.5
Noise:  8e-05  True positive:  0.5

Noise:  0.0  True positive:  0.5
Noise:  1e-05  True positive:  0.5
Noise:  2e-05  True positive:  0.5
Noise:  3e-05  True positive:  0.5
Noise:  4e-05  True positive:  0.5
Noise:  5e-05  True positive:  0.4
Noise:  6e-05  True positive:  0.5
Noise:  7e-05  True positive:  0.4
Noise:  8e-05  True positive:  0.5

Noise:  0.0  True positive:  0.35
Noise:  1e-05  True p

In [ ]:
optimum_noise = 0.00002
optimum_noise

2e-05

Upon observing the true positive rates for a bunch of noise values, the optimum noise value to get true positive rate 50% is 0.00002

### Step 7
Explain how one can improve the true positive rate in the presence of such a noise.

One of the ways to improve the true positive rate can be to time each instruction of the prover function and check the execution time of each instruction such as `sha256()`, `update()`, `if`, `else`, `append`, `encode()`, `zip()` etc. and then deterministically find a threshold. It would need modification of the prover and malicious prover function such that the correct computation instructions are executed in a similar section of the code.

Example, everything suspicious and additional should happen **after** the computation of hash, and nothing should be before that.